In [ ]:
import pandas as pd
from datetime import datetime, date
from pandas import json_normalize

pd.set_option('display.max_columns', None)

In [ ]:
df_author_raw = catalog.load('raw/openalex/author#parquet')

                    INFO     Loading data from raw/openalex/author#parquet (ParquetDataset)...  ]8;id=273812;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=27154;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

# Profiling

In [ ]:
df_author_raw.dtypes


id                           object
orcid                        object
display_name                 object
display_name_alternatives    object
works_count                   int64
cited_by_count                int64
summary_stats                object
ids                          object
affiliations                 object
last_known_institutions      object
topics                       object
topic_share                  object
x_concepts                   object
counts_by_year               object
works_api_url                object
updated_date                 object
created_date                 object
extract_datetime             object
dtype: object

In [ ]:
df_author_raw

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date,extract_datetime
0,https://openalex.org/A5100603203,https://orcid.org/0000-0003-4610-8714,Christian Weber,"[Christian Weber, C WEBER, C Weber, C. H. Webe...",1852,98141,"{'2yr_mean_citedness': 4.141414141414141, 'h_i...",{'openalex': 'https://openalex.org/A5100603203...,"[{'institution': {'country_code': 'DE', 'displ...","[{'country_code': 'US', 'display_name': 'Bosto...","[{'count': 281, 'display_name': 'Atheroscleros...",[{'display_name': 'Atherosclerosis and Cardiov...,"[{'display_name': 'Cancer', 'id': '121608353',...","[{'cited_by_count': 0, 'oa_works_count': 0, 'w...",https://api.openalex.org/works?filter=author.i...,2025-11-03T23:22:51.186590,2016-06-24,2025-11-23
1,https://openalex.org/A5110879320,None,Víctor Hugo,"[Víctor Hugo, Hugo Victor, Hugo, Victor, Victo...",1610,696,"{'2yr_mean_citedness': 0.5333333333333333, 'h_...",{'openalex': 'https://openalex.org/A5110879320...,"[{'institution': {'country_code': 'US', 'displ...",[],"[{'count': 170, 'display_name': 'French Litera...",[{'display_name': 'French Literature and Criti...,"[{'display_name': 'Public economics', 'id': '1...","[{'cited_by_count': 1, 'oa_works_count': 0, 'w...",https://api.openalex.org/works?filter=author.i...,2025-11-01T23:18:31.786418,2016-06-24,2025-11-23
2,https://openalex.org/A5102821140,https://orcid.org/0000-0002-8659-5767,C. Weber,"[C. Weber, A. Cecchi, AF Webb, C Weber, Christ...",886,21405,"{'2yr_mean_citedness': 5.883620689655173, 'h_i...",{'openalex': 'https://openalex.org/A5102821140...,"[{'institution': {'country_code': 'US', 'displ...","[{'country_code': 'US', 'display_name': 'Brook...","[{'count': 834, 'display_name': 'Particle phys...",[{'display_name': 'High-Energy Particle Collis...,"[{'display_name': 'Particle physics', 'id': '1...","[{'cited_by_count': 0, 'oa_works_count': 0, 'w...",https://api.openalex.org/works?filter=author.i...,2025-11-01T23:18:31.786418,2016-06-24,2025-11-23
3,https://openalex.org/A5083202579,https://orcid.org/0000-0003-2187-4033,Julio Bobes,"[Julio Bobes, Bobes García J, Bobes J, Bobes-G...",874,30177,"{'2yr_mean_citedness': 4.775280898876405, 'h_i...",{'openalex': 'https://openalex.org/A5083202579...,"[{'institution': {'country_code': 'US', 'displ...","[{'country_code': 'ES', 'display_name': 'Unive...","[{'count': 271, 'display_name': 'Schizophrenia...",[{'display_name': 'Schizophrenia research and ...,"[{'display_name': 'Gene', 'id': '104317684', '...","[{'cited_by_count': 4, 'oa_works_count': 1, 'w...",https://api.openalex.org/works?filter=author.i...,2025-11-02T23:13:26.583910,2016-06-24,2025-11-23
4,https://openalex.org/A5004749083,https://orcid.org/0000-0002-3467-9416,Y. Maeno,"[Y. Maeno, Maeno Yoshiteru, Maeno, Y, Maeno, Y...",866,34970,"{'2yr_mean_citedness': 4.457142857142857, 'h_i...",{'openalex': 'https://openalex.org/A5004749083...,"[{'institution': {'country_code': 'JP', 'displ...","[{'country_code': 'JP', 'display_name': 'Kyoto...","[{'count': 670, 'display_name': 'Advanced Cond...",[{'display_name': 'Advanced Condensed Matter P...,"[{'display_name': 'Piezoelectricity', 'id': '1...","[{'cited_by_count': 16, 'oa_works_count': 0, '...",https://api.openalex.org/works?filter=author.i...,2025-10-27T03:11:17.418693,2016-06-24,2025-11-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://openalex.org/A5062011227,None,José María Herranz de la Casa,"[José María Herranz de la Casa, Herranz de la ...",129,435,"{'2yr_mean_citedness': 3.0, 'h_index': 11, 'i1...",{'openalex': 'https://openalex.org/A5062011227...,"[{'institution': {'country_code': 'ES', 'displ...","[{'country_code': 'ES', 'display_name': 'Unive...","[{'count': 35, 'display_name': 'Advertising an...",[{'display_name': 'Communication and COVID-19 ...,"[{'display_name': 'Finance', 'id': '10138342

# Nodo

In [ ]:
def openalex_load_author_institution_year(df: pd.DataFrame)-> pd.DataFrame:

    # Selecciono columna con id de author y afiliación
    df_author = df.loc[:, ['id', 'affiliations']]
    df_author = df_author.convert_dtypes()

    # Proceso columna 'affiliations'
    df_author = df_author.explode('affiliations').reset_index(drop=True)
    affiliation_expanded = pd.json_normalize(df_author["affiliations"])
    affiliation_expanded = affiliation_expanded.loc[:,['institution.id','years']]

    df_author2affiliation = pd.concat([df_author, affiliation_expanded], axis=1)
    df_author2affiliation.drop(columns=["affiliations"], inplace=True)
    df_author2affiliation = df_author2affiliation.explode('years')

    df_author2affiliation.rename(columns={'institution.id':'institution_id'}, inplace=True)
    df_author2affiliation = df_author2affiliation.convert_dtypes()

    df_author2affiliation['load_datetime'] = pd.to_datetime(datetime.today())
    
    return df_author2affiliation



# Ejecuto nodo

In [ ]:
df_author = openalex_load_author_institution_year(df_author_raw)

# Resultados

In [ ]:
df_author

,id,institution_id,years,load_datetime
0,https://openalex.org/A5100603203,https://openalex.org/I102335020,2017,2025-11-23 13:39:58.767180
0,https://openalex.org/A5100603203,https://openalex.org/I102335020,2015,2025-11-23 13:39:58.767180
0,https://openalex.org/A5100603203,https://openalex.org/I102335020,2014,2025-11-23 13:39:58.767180
1,https://openalex.org/A5100603203,https://openalex.org/I103163165,2024,2025-11-23 13:39:58.767180
2,https://openalex.org/A5100603203,https://openalex.org/I107257983,1996,2025-11-23 13:39:58.767180
...,...,...,...,...
4158,https://openalex.org/A5030267796,https://openalex.org/I53241121,2011,2025-11-23 13:39:58.767180
4158,https://openalex.org/A5030267796,https://openalex.org/I53241121,2010,2025-11-23 13:39:58.767180
4158,https://openalex.org/A5030267796,https://openalex.org/I53241121,2009,2025-11-23 13:39:58.767180
4158,https://openalex.org/A5030267796,https://openalex.org/I53241121,2008,2025-11-23 13:39:58.767180
